In [34]:
import numpy as np

data = np.random.rand(2, 3, 4)
data

array([[[0.43219313, 0.43819836, 0.88376495, 0.24624543],
        [0.36893658, 0.35314718, 0.65310365, 0.31627816],
        [0.9240937 , 0.74363723, 0.07594521, 0.24081878]],

       [[0.00432076, 0.35138105, 0.54549914, 0.87260439],
        [0.42822147, 0.63953437, 0.884968  , 0.44001708],
        [0.17099976, 0.3738972 , 0.52043915, 0.48967166]]])

In [35]:
import pyarrow as pa

tensor = pa.Tensor.from_numpy(data)
with pa.OSFile('data.parquet', 'wb') as sink:
    pa.ipc.write_tensor(tensor, sink)

In [36]:
with pa.OSFile('data.parquet', 'rb') as source:
    tensor = pa.ipc.read_tensor(source)
tensor.to_numpy()

array([[[0.43219313, 0.43819836, 0.88376495, 0.24624543],
        [0.36893658, 0.35314718, 0.65310365, 0.31627816],
        [0.9240937 , 0.74363723, 0.07594521, 0.24081878]],

       [[0.00432076, 0.35138105, 0.54549914, 0.87260439],
        [0.42822147, 0.63953437, 0.884968  , 0.44001708],
        [0.17099976, 0.3738972 , 0.52043915, 0.48967166]]])